# CS 5540 - Group 1 - Apache Spark Assignment

This is the submission document for our programming assignment over Apache Spark. 

The submission was written as a Jupyter notebook but will be exported to a PDF for submission. We can provide the GitHub repo or the original Jupyter notebook if requested.

In [3]:
import pyspark as spark

## Table of Contents

1. [Team Members](#team-members)

## Team Members

This assignment was completed by the following team members (Group 1):

- Ayushman Das
- Koti Paruchuri
- Odai Athamneh
- Scott Brunton
- Varshith Thota

## Question 1

Question 1 is as follows: 

> Given file (`/data/shakespeare-1.txt`) contains the scenes from Shakespeare’s plays. You may use this 
file as an input dataset to identify the following notes for a student of Classical Drama.

In [4]:
df = spark.read.txt("data/shakespeare-1.txt")
df.show()

AttributeError: module 'pyspark' has no attribute 'read'

### Question 1.1

> How many different countries are mentioned in the whole file? (Regardless of 
how many times a single country is mentioned, this country only contributes as a single entry). 